In [ ]:
#| include: false
#skip
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

In [ ]:
#| default_exp layers

In [ ]:
#| export
from fastai.imports import *
from fastai.torch_imports import *
from fastai.torch_core import *
from fastai.layers import *

In [ ]:
#| export
from fastai.text.models.awdlstm import EmbeddingDropout, RNNDropout

In [ ]:
#| include: false
from nbdev.showdoc import *

## Basic manipulations and resizing

One can easily create a beautiful layer with minimum boilerplate using fastai utilities. We will show a simple example here. For details and extensive illustrations please refer to [decorated fastai layers](https://docs.fast.ai/layers.html#Basic-manipulations-and-resize).

An easy way to create a pytorch layer for a simple `func`

In [ ]:
def _add2(x): return x+2
tst = Lambda(_add2)
x = torch.randn(10,20)
test_eq(tst(x), x+2)
tst2 = pickle.loads(pickle.dumps(tst))
test_eq(tst2(x), x+2)

## BatchNorm layers

In [ ]:
#| export
class LinBnDrop(nn.Sequential):
    "Module grouping `BatchNorm1d`, `Dropout` and `Linear` layers"
    def __init__(self, n_in, n_out=None, bn=True, ln=True, p=0., act=None, lin_first=False):
        layers = [BatchNorm(n_out if ln and lin_first else n_in, ndim=1)] if bn else []
        if p != 0: layers.append(nn.Dropout(p))
        lin = [nn.Linear(n_in, n_out, bias=not bn)] if ln else []
        if ln and act is not None: lin.append(act)
        layers = lin+layers if lin_first else layers+lin
        super().__init__(*layers)

The `BatchNorm` or the `Linear` layer is skipped if `bn=False` or `ln=False`, as is the dropout if `p=0`. Optionally, you can add an activation for after the linear layer with act.

In [ ]:
tst = LinBnDrop(10, 20)
list(tst.children())

[BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 Linear(in_features=10, out_features=20, bias=False)]

In [ ]:
tst = LinBnDrop(10, 20, ln=False, p=0.02)
tst

LinBnDrop(
  (0): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Dropout(p=0.02, inplace=False)
)

The `LinBnDrop` layer ia not going to add an activation if `ln` is `False`:

In [ ]:
tst = LinBnDrop(10, 20, ln=False, p=0.02, act=nn.ReLU(inplace=True))
tst

LinBnDrop(
  (0): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Dropout(p=0.02, inplace=False)
)

## Attention Layers

In [ ]:
#| export
class XMLAttention(Module):
    "Compute label specific attention weights for each token in a sequence"
    def __init__(self, n_lbs, emb_sz, embed_p):
         store_attr('n_lbs,emb_sz,embed_p')
         self.lbs_weight = nn.Embedding(n_lbs, emb_sz)
         self.lbs_weight_dp = EmbeddingDropout(self.lbs_weight, embed_p)
         self.lbs_weight.weight.data.normal_(0, 0.01)   
         self.input_dp = RNNDropout(0.02)

    def forward(self, x):
        lbs_emb = self.lbs_weight(torch.arange(self.n_lbs, device=x.device))
        # x_dp = self.input_dp(x)
        attn_wgts = F.softmax(x @ lbs_emb.transpose(0,1), dim=1)
        return attn_wgts.transpose(1,2) @ x
    

In [ ]:
x = torch.randn(3,2,4).cuda()
x.device

device(type='cuda', index=0)

In [ ]:
x.size()

torch.Size([3, 2, 4])

In [ ]:
x.new_full((10,), 4)

tensor([4., 4., 4., 4., 4., 4., 4., 4., 4., 4.], device='cuda:0')

In [ ]:
enc = nn.Embedding(10, 7, padding_idx=1)

In [ ]:
pd.DataFrame(enc.weight)

,0,1,2,3,4,5,6
0,-0.354542,0.120183,-1.069463,0.033049,1.497093,-0.372787,-0.746395
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-0.197547,-2.609504,-0.098179,0.858028,-0.698687,-0.486721,1.989277
3,1.055056,-0.886635,-1.460489,0.671491,-0.597740,-0.821621,-0.211107
4,0.406442,1.518790,1.492673,2.708379,-2.742780,0.964204,-1.016047
5,0.858705,0.135087,1.438656,0.443722,-0.363109,0.084536,0.378795
6,0.561239,-0.876000,1.173250,0.168502,-0.885682,0.164763,-1.404446
7,-1.037571,-0.781387,-1.035010,2.405637,0.795765,0.292884,-0.282768
8,0.285968,-1.314268,-0.268181,-0.687591,1.183172,0.380366,-0.479281
9,0.177469,0.942547,0.259131,0.531563,-0.474286,-1.260258,0.363809


In [ ]:
enc_dp = EmbeddingDropout(enc, 0.5)

In [ ]:
tst_inp = torch.randint(0,10,(8,))

In [ ]:
tst_inp

tensor([5, 8, 9, 1, 9, 0, 8, 7])

In [ ]:
pd.DataFrame(enc(tst_inp), index=to_np(tst_inp))

,0,1,2,3,4,5,6
5,0.858705,0.135087,1.438656,0.443722,-0.363109,0.084536,0.378795
8,0.285968,-1.314268,-0.268181,-0.687591,1.183172,0.380366,-0.479281
9,0.177469,0.942547,0.259131,0.531563,-0.474286,-1.260258,0.363809
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.177469,0.942547,0.259131,0.531563,-0.474286,-1.260258,0.363809
0,-0.354542,0.120183,-1.069463,0.033049,1.497093,-0.372787,-0.746395
8,0.285968,-1.314268,-0.268181,-0.687591,1.183172,0.380366,-0.479281
7,-1.037571,-0.781387,-1.035010,2.405637,0.795765,0.292884,-0.282768


In [ ]:
tst_out = enc_dp(tst_inp)
tst_out.shape

torch.Size([8, 7])

In [ ]:
pd.DataFrame(tst_out, index=tst_inp.numpy())

,0,1,2,3,4,5,6
5,1.71741,0.270174,2.877312,0.887444,-0.726219,0.169072,0.757589
8,0.00000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000
9,0.00000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000
1,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.00000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000
0,-0.00000,0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000
8,0.00000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000
7,-0.00000,-0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000


In [ ]:
vocab = torch.arange(10)

In [ ]:
pd.DataFrame(enc_dp(torch.arange(10)), vocab.numpy())

,0,1,2,3,4,5,6
0,-0.709084,0.240366,-2.138925,0.066098,2.994186,-0.745574,-1.492791
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-0.000000,-0.000000,-0.000000,0.000000,-0.000000,-0.000000,0.000000
3,0.000000,-0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.000000
4,0.812884,3.037580,2.985346,5.416758,-5.485560,1.928409,-2.032093
5,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000
6,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000
7,-2.075141,-1.562775,-2.070020,4.811273,1.591530,0.585767,-0.565536
8,0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000
9,0.354938,1.885094,0.518262,1.063126,-0.948572,-2.520516,0.727617


In [ ]:
import nbdev; nbdev.nbdev_export()